# Predicting german doctor ratings with a BERT model
## Fine tuning of a pretrained **Hugging Face** transfomer
In this notebook we will be looking at the fine-tuning process of a BERT model that was previously pre-trained on a large german text corpus.

You will build a classifier to predict doctor ratings from patients' text comments. Along the way you will load the dataset in the approprate format for a transformer, perform a perliminary data analysis, implement the neccessary layers in PyTorch and finally train the model.

**Important**: Later you will use Tensorboard to track the training progress. Sometimes it can't properly be loaded if you are employing an adblocker. So better turn it off before you get started.

A detailed description of the **German language reviews of doctors by patients 2019** dataset can be found [here](https://data.world/mc51/german-language-reviews-of-doctors-by-patients)


For the feature creation and the modeling, we will use the [**Hugging Face**](https://huggingface.co/) implementation of transformers for PyTorch.

Credits: This notebook and its implementation is heavily influenced by the [data-drive](https://data-dive.com/) *Natural Language Processing of German texts* blog post

In [ ]:
!pip install -U transformers==4.42.4

In [ ]:
import re
import tqdm
from datetime import datetime

import pandas as pd
import numpy as np
import seaborn as sns

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter


from sklearn import metrics
from sklearn.model_selection import train_test_split

tqdm.tqdm.pandas()
sns.set_style('whitegrid')

pd.options.display.max_colwidth = 600
pd.options.display.max_rows = 400

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'{device=}')

## Preparing the **German language reviews of doctors by patients 2019** dataset

In [ ]:
!wget -O reviews.zip https://query.data.world/s/v5xl53bs2rgq476vqy7cg7xx2db55y
!unzip reviews.zip

### Data loading and cleaning
The dataset contains patients' text comments in german and a coresponding rating ranging from 1-6 (from good to bad).
We will turn the rating into a binary grade:
- ratings from 1 to 2 will be considered "good" grades (0)
- ratings from 5 to 6 will be considered "bad" grades (1)

Entries with ratings inbetween and reviews shorter than 20 characters will be discarded.

Then we perform some rudimentary clean up by removing taggs and special characters

In [ ]:
RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
RE_TAGS = re.compile(r"<[^>]+>")
RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

def clean_text(text):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
    """

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
    return text

In [ ]:
# read data from csv
reviews = pd.read_csv("german_doctor_reviews.csv")

# Create binary grade, class 1-2 or 5-6  = good or bad
reviews["grade"] = np.nan
reviews.loc[reviews["rating"] <= 2, "grade"] = 0
reviews.loc[reviews["rating"] >= 5, "grade"] = 1

reviews = reviews[reviews["comment"].str.len() > 20]
reviews = reviews.dropna(axis="index", subset=["grade"])

reviews = reviews[["comment", "grade"]]

reviews["comment"] = reviews["comment"].progress_map(clean_text)

### Limiting the dataset size
Here we limit the size of the dataset to accelerate the training iterations. Once everything is correctly set up we should traing on the entire dataset

In [ ]:
reviews = reviews.iloc[:10_000]

### Dataset analysis

##### Text inspection
We successfully loaded the dataset into memory and performed some modification(and temporarly reduced its size ).
Now we should ensure that the data is well structured and suitable for our training

Have the grades correctly be assigned to the comments? Are there some outliers that do not make sense?

In [ ]:
reviews.head(5)

How does the distribution of the text length look like? How much of the comments will be truncated if the supported sequence length of the model is shorter?

A good way to quickly get a grasp of the distribution is to visuzalize it as bars in a histogram plot

**NOTE**: The data is loaded into a Pandas DataFrame. Use the built-in functionality compute aggregates over the 'text' column and directly plot it

In [ ]:
##########################
## YOUR CODE HERE START ##
##########################

# compute the distribution over the text lengths in the dataset
# plot the aggregation as histogram-bar-plot

reviews['comment']....

##########################
## YOUR CODE HERE END ##
##########################

##### Label inspection

Use the text data analysis techniques that you have learned so far to get a better feeling of our data.
How does the distribution of grade look like, now that we have derived them from the reviewer rating?

The best way to quickly get a grasp of the distribution is to visuzalize it as a bar-plot or piec-chart

**NOTE**: The data is loaded into a Pandas DataFrame. Use the built-in functionality compute aggregates over the 'grade' column and directly plot it.

Is the data distribution skewed? If so, why and how could this influence the training procedure?


In [ ]:
##########################
## YOUR CODE HERE START ##
##########################

# compute the distribution over the grades in the dataset
# plot the aggregation as bar-plot

reviews['grade']....

##########################
## YOUR CODE HERE END ##
##########################

## Setting up the encoder model
HuggingFace's transfomer library provides pre-configured text tokenizers and pre-trained models that can be convininetly loaded either from local resources or from [HunggingFace's model hub](https://huggingface.co/models)

Here we will be using the [**"bert-base-german-cased"**](https://huggingface.co/bert-base-german-cased) model that was pretrained on German Wikipedia dump (6GB of raw txt files), the OpenLegalData dump (2.4 GB) and news articles (3.6 GB).

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

### Text tokenization
Here we tokenize each text review into input_ids: This corresponds to the tokens in the inputs converted into IDs.

The tokenizer takes care of
- extending the token sequence with the special tokens [CLS] and [SEP]
- padding shorter sequences to the required length
- truncating long sequences to a max length


In [ ]:
MAXLEN = 192

def tokenize(review):
    encoded = tokenizer.encode_plus(
        text=review,
        add_special_tokens=True,  # Add `[CLS]` and `[SEP]`
        max_length=MAXLEN,  # Max length to truncate/pad
        padding='max_length',  # Pad sentence to max length
        return_attention_mask=True,  # Return attention mask
        return_token_type_ids=False,
        truncation=True, )

    return encoded['input_ids'], encoded['attention_mask']

In [ ]:
# Tokenize all reviews
input_ids = []
attention_masks = []

for review in tqdm.tqdm(reviews['comment']):
    ids, mask = tokenize(review)
    input_ids.append(ids)
    attention_masks.append(mask)

input_ids = np.array(input_ids)
attention_masks = np.array(attention_masks)

Now lets split the data into two datasets: one for training and one for testing.

We will use 75% for training and 25% for testing

In [ ]:
train_ids, test_ids, train_masks, test_masks, train_labels, test_labels = train_test_split(
    input_ids,
    attention_masks,
    reviews["grade"],
    random_state=1,
    test_size=0.25,
    shuffle=True
)
print(f"Train set: {len(train_ids)}\nTest set: {len(test_ids)}")

In [ ]:
# Convert to PyTorch tensors
train_ids = torch.tensor(train_ids, dtype=torch.long)
test_ids = torch.tensor(test_ids, dtype=torch.long)
train_masks = torch.tensor(train_masks, dtype=torch.long)
test_masks = torch.tensor(test_masks, dtype=torch.long)
train_labels = torch.tensor(train_labels.values, dtype=torch.float)
test_labels = torch.tensor(test_labels.values, dtype=torch.float)

# Create PyTorch datasets
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
test_dataset = TensorDataset(test_ids, test_masks, test_labels)

# Define hyperparameters
BATCH_SIZE = 16

# Create data loaders
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE
)

## Model creation
Here we build our model using a pre-trained version of a BERT model.
We will be using the output of encoder and input into a simple FFNN-stack. The last layer will translate the hidden states into a binary signal (class-probabilities bewtween 0 and 1).

The documentation of the employed [**BertModel**](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel) class describes the returned output of the BERT model.

**NOTE**:
- the special token, [CLS], is appended to the start of the inputs. [CLS] stands for classifier token. The embedding of this token is summarizing the entire input i.e. the input-sequence
- in the case of a classification problem, new additional layers on top of the BERT model would use this [CLS] token.
- the `BertModel` returns an output that contains: `last_hidden_state` and `pooler_output`
 - `last_hidden_state` is a tensor that contains the embeddings of all output tokens i.e. for the [CLS] and all sequence token
 - `pooler_output` is a tensor that only contains the embedding of the [CLS]
 - use the `pooler_output` returned by the BertModel for the subsequent layers


**NOTE**: This is a binary classification problem. Think about the correct activation function for this kind of task.

In [ ]:
##########################
## YOUR CODE HERE START ##
##########################

class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-german-cased")

        hidden_size = self.bert.config.hidden_size

        # Create a linear layer that maps from the embedding dimension to the output dimension
        self.classifier = ....

        # Create an activation layer with the appropriate function
        self.sigmoid = ....

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output

        # pass the cls_embedding to the classifier layer
        x = ....

        # pass the result to the activation layer
        output = ....

        return output

##########################
## YOUR CODE HERE END ##
##########################


In [ ]:
# Initialize model
model = BertClassifier()
model.to(device)

# Print model summary
print(model)

#### Finalizing the model
Now that we successfully created our model, we have to finalize it by defining the loss that has to be minimized and which optimizier we want to use.

The PyTorch documentation lists all available [losses](https://docs.pytorch.org/docs/stable/nn.html#loss-functions) and [optimizers](https://docs.pytorch.org/docs/stable/optim.html) you should choose from.

**NOTE**: This is a binary classification problem. Think about the correct loss function for this kind of problem.

**NOTE**: Make sure to initialize your optimizer with the learning rate that was defined above.

In [ ]:
# Define hyperparameters
EPOCHS = 2
LEARNING_RATE = 1e-5

##########################
## YOUR CODE HERE START ##
##########################

# Create an instance of an Adam optimizer with the learning rate defined above
# Define the loss to be minimized in this binary classification problem

optimizer = ....
criterion = ....

##########################
## YOUR CODE HERE END ##
##########################

## Model training
Now we are ready to fine-tune the BERT model on our dataset
We will be monitoring the training process using tensorboard. Once the training is launched you will be able to inspect logged metrics and track the progress.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
# Training function
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch in tqdm.tqdm(dataloader):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device).unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)

        total_loss += loss.item()

        # Calculate accuracy
        predictions = (outputs > 0.5).float()
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

        loss.backward()
        optimizer.step()

    return total_loss / len(dataloader), correct_predictions / total_predictions

# Evaluation function
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device).unsqueeze(1)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)

            total_loss += loss.item()

            # Calculate accuracy
            predictions = (outputs > 0.5).float()
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

    return total_loss / len(dataloader), correct_predictions / total_predictions


In [ ]:
# Training loop
best_val_loss = float('inf')
history = {'epoch': [], 'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

writer = SummaryWriter(f'logs/{datetime.now()}')

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')

    # Train
    train_loss, train_acc = train_epoch(model, train_dataloader, optimizer, criterion, device)

    # Evaluate
    val_loss, val_acc = evaluate(model, test_dataloader, criterion, device)

    # Log metrics
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/validation', val_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)
    writer.add_scalar('Accuracy/validation', val_acc, epoch)

    # Save history
    history['epoch'].append(epoch)
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}')
    print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pt')
        print("Saved best model!")

    print('-' * 60)

### Training metrics
Lets inspect the training history to see how the losses and metrics behaved during training

In [ ]:
history_df = pd.DataFrame(history).set_index('epoch')
history_df

In [ ]:
history_df.plot()

## Model evaluation
Lets have a look how well the model is preforming on our test dataset.

In [ ]:
# Load best model
model.load_state_dict(torch.load('best_model.pt'))
model.eval()

# Get predictions
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids, attention_mask)
        predictions = (outputs > 0.5).float().squeeze().cpu().numpy()

        all_predictions.extend(predictions)
        all_labels.extend(labels.cpu().numpy())

# Generate classification report
report = metrics.classification_report(y_true=all_labels,
                                      y_pred=all_predictions)
print(report)

## Congratulation
You have trained and evaluated a text classifier based on a pre-trained BERT transformer using PyTorch!